<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_preprocessing/News_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import json, os, re

from tqdm import tqdm
from datetime import datetime

# **데이터 불러오기**

In [54]:
path = '/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/00. Data/07. 뉴스'
files = os.listdir(path)

news = []
for file in tqdm(files):
    with open(path+'/'+file, 'rb') as f:
        for item in json.load(f):
            news_content = {'date':0, 'title':0, 'text':0}
            news_content['date'] = item['date']
            news_content['title'] = item['title']
            news_content['text'] = item['text']
            news.append(news_content)

100%|██████████| 5/5 [01:03<00:00, 12.73s/it]


In [155]:
news_df = pd.DataFrame(news)
news_df.head(), len(news_df)

(         date  ...                                               text
 0  2019-02-01  ...  EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중  (브뤼셀·서울=연합뉴스)...
 1  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
 2  2019-02-01  ...  (뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
 3  2019-02-01  ...  방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것”      ...
 4  2019-02-01  ...  [세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...
 
 [5 rows x 3 columns], 605268)

# **데이터 전처리**

In [156]:
news_df = news_df.drop_duplicates(['text']) # 본문 중복 제거
news_df = news_df.reset_index()
news_df = news_df.drop(['index'], axis=1)
len(news_df)

372384

In [157]:
news_df = news_df.fillna("")
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 (브뤼셀·서울=연합뉴스)...
1,2019-02-01,[인사]산업통상자원부,[세종=이데일리 김형욱 기자] ◇실장급 임용 △신통상질서전략실장 김승호 ◇국장급 승...
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,(뉴욕=연합뉴스) 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증...
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” ...
4,2019-02-01,[인사]한국로봇산업진흥원,[세종=이데일리 김형욱 기자] ◇부서장 △경영지원실장 김종헌 ◇팀장 △기획팀장 문상...


In [158]:
news_df['text'] = news_df.text.apply(lambda x : x.replace('.\n', '').replace('\\t', '').replace('\n', ' '))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\[[가-히a-zA-Z=· @.]*\]', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\([가-히a-zA-Z=· @.]*\)', '', x))
news_df['text'] = news_df.text.apply(lambda x : re.sub('\<[가-히a-zA-Z=· @.]*\>', '', x))
news_df['text'] = news_df.text.apply(lambda x : x.split('.'))
news_df['text'] = news_df.loc[news_df['text'].apply(lambda x:len(x)) >= 10, "text"] # 문서 길이 10 이상인 뉴스
news_df.head()

,date,title,text
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영...
1,2019-02-01,[인사]산업통상자원부,NaN
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,"[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다,..."
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” ...
4,2019-02-01,[인사]한국로봇산업진흥원,NaN


# **데이터 라벨링**

In [159]:
news_df['new_date'] = 0
news_df['new_date'] = news_df.date.apply(lambda x : str(datetime.strptime(x, "%Y-%m-%d"))[:10])
news_df.head()

,date,title,text,new_date
0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영...,2019-02-01
1,2019-02-01,[인사]산업통상자원부,NaN,2019-02-01
2,2019-02-01,美 1월 일자리 30만4천개 증가…셧다운에 실업률 4%로 상승,"[ 이귀원 특파원 = 미국의 지난달 일자리가 시장 기대치를 웃도는 증가세를 보였다,...",2019-02-01
3,2019-02-01,"김현종 “1월 수출 감소, 마음 무겁지만 낙담 일러”",[방미 중 페이스북 계정 통해 전해 “귀국 후 상황 점검 애로 해소할 것” ...,2019-02-01
4,2019-02-01,[인사]한국로봇산업진흥원,NaN,2019-02-01


In [160]:
path_2019 = '/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/02. Preprocessing_Data/Data_Set/2019_data.csv'
path_2020 = '/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/02. Preprocessing_Data/Data_Set/2020_data.csv'

In [161]:
df_2019 = pd.read_csv(path_2019, index_col=0)
df_2020 = pd.read_csv(path_2020, index_col=0)
df_2019.head(), df_2020.head()

(      2019-02-01  2019-02-02  2019-02-03  ...  2019-05-29  2019-05-30  2019-05-31
 숙박       1475301     1662554     1519794  ...     2228258     2186711     2577217
 레저용품      925726     1452418     1088861  ...      850541      769525      922809
 레저업소      556821      453984      377059  ...      459924      471977      555272
 문화취미      221697      303195      241879  ...      198939      234099      216266
 가구         20244       64324       10250  ...       55866       40236       24945
 
 [5 rows x 120 columns],
       2020-02-01  2020-02-02  2020-02-03  ...  2020-05-29  2020-05-30  2020-05-31
 숙박       1964575     1615471     1172604  ...      460923      894889      916030
 레저용품      987175      909642      505186  ...      921117     1654434     1457399
 레저업소      534261      409356      601070  ...      497238      418798      307580
 문화취미      183967      112281      138104  ...      149152      168967      138921
 가구        121417       84588       67909  ...       90561  

## **2019년 라벨링**

In [162]:
sum_2019 = pd.DataFrame(df_2019.sum(axis=0), columns=['총합'])
sum_2019.head()

,총합
2019-02-01,38246556
2019-02-02,42246932
2019-02-03,31092889
2019-02-04,25671360
2019-02-05,14152211


In [171]:
# 전체 매출량 차이 라벨링(전날 대비 상승, 하강, 변화없음)
sum_2019['diff'] = 0
sum_2019['label'] = 0
for i in range(0, len(sum_2019)-1):
    sum_2019['diff'][i+1] = float(sum_2019['총합'][i+1] - sum_2019['총합'][i])
    if sum_2019['diff'][i+1] < 0:
        sum_2019['label'][i+1] = -1
    elif sum_2019['diff'][i+1] > 0:
        sum_2019['label'][i+1] = 1
    else:
        sum_2019['label'][i+1] = 0
sum_2019.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,총합,diff,label,new_date
2019-02-01,38246556,0,0,2019-02-01
2019-02-02,42246932,4000376,1,2019-02-02
2019-02-03,31092889,-11154043,-1,2019-02-03
2019-02-04,25671360,-5421529,-1,2019-02-04
2019-02-05,14152211,-11519149,-1,2019-02-05


In [173]:
# 날짜 데이트화
sum_2019['new_date'] = str(0)
date_list = sum_2019.index.to_list()
for i in range(len(date_list)):
    sum_2019['new_date'][i] = str(datetime.strptime(date_list[i], "%Y-%m-%d"))[:10]

sum_2019.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,총합,diff,label,new_date
2019-02-01,38246556,0,0,2019-02-01
2019-02-02,42246932,4000376,1,2019-02-02
2019-02-03,31092889,-11154043,-1,2019-02-03
2019-02-04,25671360,-5421529,-1,2019-02-04
2019-02-05,14152211,-11519149,-1,2019-02-05


In [174]:
diff = sum_2019[['new_date', 'label']]
diff.head()

,new_date,label
2019-02-01,2019-02-01,0
2019-02-02,2019-02-02,1
2019-02-03,2019-02-03,-1
2019-02-04,2019-02-04,-1
2019-02-05,2019-02-05,-1


In [175]:
news_wt_diff = pd.merge(diff, news_df, how = 'left', on='new_date')
news_wt_diff = news_wt_diff.sort_values(by = ['date'], axis=0, ascending=True)
news_wt_diff = news_wt_diff.reset_index()
news_wt_diff = news_wt_diff.drop(['index'], axis=1)
news_wt_diff.head()

,new_date,label,date,title,text
0,2019-02-01,0,2019-02-01,폴란드산 '병든 소' 유럽 13개국서 유통…EU 긴급조사 나서(종합),[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영...
1,2019-02-01,0,2019-02-01,예천군의원 2명 제명 확정…군민 `셀프 징계` 반발,"[ 예천군의회가 1일 박종철 의원과 권도식 의원에 대한 제명안을 통과시켰다, 예..."
2,2019-02-01,0,2019-02-01,출석하는 안희정,NaN
3,2019-02-01,0,2019-02-01,"'부하 성폭력' 안희정, 항소심 선고 출석…'묵묵부답'",[방청객 100여명 방청 대기…'안희정은 유죄다' 구호 외치는 사람들도 지위...
4,2019-02-01,0,2019-02-01,아직 한산한 귀성길,NaN


In [176]:
data = news_wt_diff[['new_date', 'text', 'label']]
data = data.dropna()
data = data.reset_index()
data.head()

,index,new_date,text,label
0,0,2019-02-01,[EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영...,0
1,1,2019-02-01,"[ 예천군의회가 1일 박종철 의원과 권도식 의원에 대한 제명안을 통과시켰다, 예...",0
2,3,2019-02-01,[방청객 100여명 방청 대기…'안희정은 유죄다' 구호 외치는 사람들도 지위...,0
3,6,2019-02-01,[전 협회 사업본부장 파라솔 수익금 횡령 회장과 공모 주장 회장 협회 장악하려는 음...,0
4,13,2019-02-01,"[법무부가 쌍용자동차 복직 노동자들의 급여 등에 대한 가압류를 해제한다, 법무부는...",0


In [177]:
new_data = []

for i in tqdm(range(len(data))):
    for j in range(len(data['text'][i])):
        tmp_data = {'date' : 0, 'content' : 0, 'label' : 0}
        tmp_data['date'] = data['new_date'][i]
        tmp_data['content'] = data['text'][i][j]
        tmp_data['label'] = data['label'][i]
        new_data.append(tmp_data)

new_data[:5]

100%|██████████| 64363/64363 [01:13<00:00, 876.41it/s]


[{'content': 'EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중   김병수 특파원 김서영 기자 = 유럽 국가에 소고기를 수출해온 폴란드의 한 도축장에서 병든 소가 도축된 것으로 드러나 파문이 일고 있다',
  'date': '2019-02-01',
  'label': 0},
 {'content': '   병든 소를 도축하는 폴란드 도축장 영상    유럽연합 행정부 격인 집행위원회는 1일 폴란드에서 병든 소가 도축된 뒤 그 고기가 수출된 것과 관련해 조사에 착수했다면서 조사관들이 오는 4일 폴란드 현지에 도착할 것이라고 밝혔다',
  'date': '2019-02-01',
  'label': 0},
 {'content': '   EU 집행위 대변인은 이날 브리핑에서 폴란드 당국이 소고기 가공공장을 폐쇄하고 시장에 유통된 문제의 소고기를 추적, 회수하고 있다며 지금까지 폴란드와 12개 주변국에서 유통된 것으로 파악됐다고 말했다',
  'date': '2019-02-01',
  'label': 0},
 {'content': '   문제의 소고기가 수출된 것으로 확인된 나라는 에스토니아, 핀란드, 프랑스, 독일, 헝가리, 라트비아, 리투아니아, 포르투갈, 루마니아, 슬로바키아, 스페인, 스웨덴 등이나 바뀔 수 있다고 이 대변인은 덧붙였다',
  'date': '2019-02-01',
  'label': 0},
 {'content': '   앞서 영국 BBC 방송은 폴란드의 한 도축장에서 도축된 병든 소가 프랑스와 독일 등 주변 국가로 수출된 것으로 확인됐다면서 EU가 해당 소고기 긴급회수와 폐기 작업을 벌이고 있다고 지난달 31일 보도했다',
  'date': '2019-02-01',
  'label': 0}]

In [180]:
data_2019 = pd.DataFrame(new_data)
data_2019.head()

,date,content,label
0,2019-02-01,EU 집행위 조사관 4일 폴란드 도착…유통 소고기 회수중 김병수 특파원 김서영 ...,0
1,2019-02-01,병든 소를 도축하는 폴란드 도축장 영상 유럽연합 행정부 격인 집행위원회는...,0
2,2019-02-01,EU 집행위 대변인은 이날 브리핑에서 폴란드 당국이 소고기 가공공장을 폐쇄하고...,0
3,2019-02-01,"문제의 소고기가 수출된 것으로 확인된 나라는 에스토니아, 핀란드, 프랑스, 독...",0
4,2019-02-01,앞서 영국 BBC 방송은 폴란드의 한 도축장에서 도축된 병든 소가 프랑스와 독...,0


## **2020 라벨링**

In [181]:
sum_2020 = pd.DataFrame(df_2020.sum(axis=0), columns=['총합'])
sum_2020.head()

,총합
2020-02-01,34512297
2020-02-02,23820677
2020-02-03,26933242
2020-02-04,26949563
2020-02-05,26597881


In [182]:
# 전체 매출량 차이 라벨링(전날 대비 상승, 하강, 변화없음)
sum_2020['diff'] = 0
sum_2020['label'] = 0
for i in range(0, len(sum_2020)-1):
    sum_2020['diff'][i+1] = float(sum_2020['총합'][i+1] - sum_2020['총합'][i])
    if sum_2020['diff'][i+1] < 0:
        sum_2020['label'][i+1] = -1
    elif sum_2020['diff'][i+1] > 0:
        sum_2020['label'][i+1] = 1
    else:
        sum_2020['label'][i+1] = 0
sum_2020.head()

,총합,diff,label
2020-02-01,34512297,0,0
2020-02-02,23820677,-10691620,-1
2020-02-03,26933242,3112565,1
2020-02-04,26949563,16321,1
2020-02-05,26597881,-351682,-1


In [184]:
# 날짜 데이트화
sum_2020['new_date'] = str(0)
date_list = sum_2020.index.to_list()
for i in range(len(date_list)):
    sum_2020['new_date'][i] = str(datetime.strptime(date_list[i], "%Y-%m-%d"))[:10]

sum_2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,총합,diff,label,new_date
2020-02-01,34512297,0,0,2020-02-01
2020-02-02,23820677,-10691620,-1,2020-02-02
2020-02-03,26933242,3112565,1,2020-02-03
2020-02-04,26949563,16321,1,2020-02-04
2020-02-05,26597881,-351682,-1,2020-02-05


In [186]:
diff = sum_2020[['new_date', 'label']]
diff.head()

,new_date,label
2020-02-01,2020-02-01,0
2020-02-02,2020-02-02,-1
2020-02-03,2020-02-03,1
2020-02-04,2020-02-04,1
2020-02-05,2020-02-05,-1


In [187]:
news_wt_diff = pd.merge(diff, news_df, how = 'left', on='new_date')
news_wt_diff = news_wt_diff.sort_values(by = ['date'], axis=0, ascending=True)
news_wt_diff = news_wt_diff.reset_index()
news_wt_diff = news_wt_diff.drop(['index'], axis=1)
news_wt_diff.head()

,new_date,label,date,title,text
0,2020-02-01,0,2020-02-01,12번째 신종코로나 확진자 다녀간 CGV부천역점 임시 휴업,[CGV 부천역점이 신종 코로나바이러스 감염증 국내 열두 번째 확진자가 다녀간 것으...
1,2020-02-01,0,2020-02-01,"신종코로나 `어린이집 교사` 6번 확진자 딸, 음성 판정",NaN
2,2020-02-01,0,2020-02-01,전국 최대 규모 청도 달집태우기 올해는 못 본다,[신종코로나 사태로 전격 취소…2017년·2019년에도 취소 2016년 청도 달...
3,2020-02-01,0,2020-02-01,[윤경 변호사의 세상萬思]“모든 일엔 때가 있고 타이밍이 중요하다”,[`가랑이 기어간 치욕` 참은 한신 漢 건국 일등 공신 대장군 올라 “세상일 뜻대로...
4,2020-02-01,0,2020-02-01,[인사] 경남지방경찰청,NaN


In [188]:
data = news_wt_diff[['new_date', 'text', 'label']]
data = data.dropna()
data = data.reset_index()
data.head()

,index,new_date,text,label
0,0,2020-02-01,[CGV 부천역점이 신종 코로나바이러스 감염증 국내 열두 번째 확진자가 다녀간 것으...,0
1,2,2020-02-01,[신종코로나 사태로 전격 취소…2017년·2019년에도 취소 2016년 청도 달...,0
2,3,2020-02-01,[`가랑이 기어간 치욕` 참은 한신 漢 건국 일등 공신 대장군 올라 “세상일 뜻대로...,0
3,5,2020-02-01,['타임캡술' 앨범 한일 동시 발매… 국경·민족 넘어 노래로 화합 22년만에 재...,0
4,6,2020-02-01,[교민들 낯선 환경서 동요 없이 생활 …이상 증세 교민도 없어 경찰 130명 배치...,0


In [189]:
new_data = []
for i in tqdm(range(len(data))):
    for j in range(len(data['text'][i])):
        tmp_data = {'date' : 0, 'content' : 0, 'label' : 0}
        tmp_data['date'] = data['new_date'][i]
        tmp_data['content'] = data['text'][i][j]
        tmp_data['label'] = data['label'][i]
        new_data.append(tmp_data)
new_data[:5]

100%|██████████| 70538/70538 [01:17<00:00, 906.56it/s]


[{'content': 'CGV 부천역점이 신종 코로나바이러스 감염증 국내 열두 번째 확진자가 다녀간 것으로 확인돼 1일 임시 휴업에 들어갔다',
  'date': '2020-02-01',
  'label': 0},
 {'content': ' CGV는 이날 홈페이지 공지를 통해 임시휴업 배경을 전하며 “고객 안전을 위한 선제적 조치이니 양해를 부탁드린다”라고 밝혔다',
  'date': '2020-02-01',
  'label': 0},
 {'content': ' CGV 부천역점은 확진자 방문 사실을 확인한 직후인 이날 오후 6시30분부터 영업을 중단했다',
  'date': '2020-02-01',
  'label': 0},
 {'content': ' 오후 7시10분께 모든 고객을 퇴장시켰다', 'date': '2020-02-01', 'label': 0},
 {'content': ' CGV 측은 “혹시 모를 상황에 대비해 부천에 있는 모든 지점을 자체 방역하고 있다”고 말했다',
  'date': '2020-02-01',
  'label': 0}]

In [190]:
data_2020 = pd.DataFrame(new_data)
data_2020.head()

,date,content,label
0,2020-02-01,CGV 부천역점이 신종 코로나바이러스 감염증 국내 열두 번째 확진자가 다녀간 것으로...,0
1,2020-02-01,CGV는 이날 홈페이지 공지를 통해 임시휴업 배경을 전하며 “고객 안전을 위한 선...,0
2,2020-02-01,CGV 부천역점은 확진자 방문 사실을 확인한 직후인 이날 오후 6시30분부터 영업...,0
3,2020-02-01,오후 7시10분께 모든 고객을 퇴장시켰다,0
4,2020-02-01,CGV 측은 “혹시 모를 상황에 대비해 부천에 있는 모든 지점을 자체 방역하고 있...,0


# **데이터 저장**

In [191]:
data_2019.to_csv('/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/02. Preprocessing_Data/Data_Set/2019_news')
data_2020.to_csv('/content/drive/My Drive/머신러닝/팀 프로젝트/빅콘테스트/02. Preprocessing_Data/Data_Set/2020_news')